# Step 1: Implement the vector gradient.

Implement the vector gradient as described in the lecture slides and the assignment description in the `color_sobel_edges()` function below.  Use sobel filters to estimate partial derivatives.  Use the `color_dot_product()` function (provided) to compute the necessary dot products to obtain gxx, gyy, and gxy.  Return an array containing the gradient magnitudes for each pixel, i.e. a graident magnitude image.  Optionally, return a second array containing the gradient directions for each pixel.

As usual, the input image must be dtype `float` or `uint8`.  If it is `uint8` convert it to `float` before processing.  Leave the magnitude image ouput as dtype `float` regardless of the input's dtype.



In [ ]:
import skimage.util as util
import numpy as np


def color_dot_product(A, B):
    '''
    Element-by-element dot product in a 2D array of vectors.

    :return: An array in which index [i,j,:] is the dot product of A[i,j,:] and B[i,j,:].
    '''
    return np.sum(A.conj()*B, axis=2)



def color_sobel_edges(I):
    '''
    Finish me!
    
    :param I: 
    :return: 
    '''
    
        red, green, blue = np.moveaxis(I, -1, 0)
    # Compute horizontal and vertical derivatives of red, blue and green channels through applying sobel filters
    r = filt.sobel_h(red)
    g = filt.sobel_h(green)
    b = filt.sobel_h(blue)
    u = array([r,g,b])
    r = filt.sobel_v(red)
    g = filt.sobel_v(green)
    b = filt.sobel_v(blue)
    v = array([r,g,b])
    gxx = color_dot_product(u, u)
    gyy = color_dot_product(v, v)
    gxy = color_dot_product(u, v)


    # Calculate gradient direction (direction of the largest colour change)
    theta = 0.5 * np.arctan(2 * gxy / (gxx - gyy))

    # Calculate the magnitude of the rate of change
    fTheta = np.sqrt(0.5 * ((gxx + gyy) + (gxx - gyy) * np.cos(2 * theta) + (2 * gxy * np.sin(2 * theta))))

    return np.array([theta, fTheta])

# Step 2: Examine behavior of the kurtosis sharpness metric.

Write a function which:

* takes as input an input image, a minimum value of sigma, and a maximum value of sigma.
* applies different amounts of Gaussian blur to the original image for all integer values of sigma between the provided minimum and maximum values of sigma, inclusive. (reminder: sigma describes the standard deviation of the gaussian filter mask used to blur the image).
* For each blurred image, compute the gradient magnitude using `color_sobel_edges()`, then compute compute the kurtosis sharpness measure.  This is the log(kurtosis+3) where kurtosis is the kurtosis of the gradient magnitude image of the blurred image as described in the assignment description document.  See `scipy.stats.kurtosis()`.
* Returns a tuple consisting of the range object of sigma values used and the list of computed kurtosis values for each sigma.

Call the function using `mushroom.jpg` as the input image, a minimum sigma of 1, and a maximum sigma of 30.  Use a smaller max sigma until you are sure it's working, then increase to 30, as it can take a few minutes to do all the filtering.  Use the return values from your function to plot a line graph of gaussian blur sigma vs. blur measure (kurtosis) for the waterfall image.  Add appropriate axis labels and a descriptive title.  Sample output is provided in the assignment description document.



In [2]:

% matplotlib inline

def test_blur_measure(I, min_sigma, max_sigma):
    '''
    Finish me!
    
    :param I: 
    :param min_sigma: 
    :param max_sigma: 
    :return: 
    '''

    s = min_sigma*max_sigma
    print(s)
    k_value = np.zeros(shape = [s, s])

    for sigma in range(min_sigma, max_sigma+1):
        gauss_I = filt.gaussian(I, sigma=sigma, multichannel=False)
        gradient_magnitude = color_sobel_edges(gauss_I)
        oned_arr = np.reshape(gradient_magnitude[0], gradient_magnitude.shape[1] * gradient_magnitude.shape[2])
        K = scipy.stats.kurtosis(oned_arr)
        sharpness = np.log(K + 3)
        s = sigma - min_sigma
        k_value[s,0] = sigma
        k_value[s, 1] = K

    return(k_value)

mushroom = io.imread('mushroom.jpg')
I = util.img_as_float(mushroom)
kurtosis = test_blur_measure(I, 1, 30)
plt.imshow(kurtosis)
plt.xlabel('Sigma')
plt.ylabel('Kurtosis')
plt.title('Kurtosis values for gaussian blurred gradient magnitudes with varying sigma values for mushroom.jpg')
plt.show()

# Step 3:  Create a local blur map

Write a function which:

* takes as input an input image and a square window size (in pixels). e.g. if `window_size = 11`, this means an 11 by 11 window.
* computes the local sharpness of the input image (i.e. log(kurtosis+3)) for each tiled, non-overlapping square window of the given window size
* stores each local sharpness in an array where each entry represents one window of the input image (the size of this array can be computed by integer division of the original image dimensions by the window size)
* returns the array of local sharpnesses.

Then call the function you just wrote with `mushroom.jpg` as the input image and 100 as the window size.  Plot the returned array as an image using `plt.imshow()`.  Do not rescale this image with `vmin=` or `vmax=`, and use the default colormap (don't change it to `'gray'`).  Add a color scale bar to the figure using `plt.colorbar()`.  Sample output is provided in the assignment description document.



In [3]:
def sharpness_map(I, window_size):
    '''
    
    Finish me!
    
    :param I: 
    :param window_size: 
    :return: 
    '''

    shrp_arr = [I.shape[0]*I.shape[1]/window_size]
    for row in range(0,I.shape[0],11):
        #loop columns
        for col in range(0,I.shape[1],11):#enumerate(row):
            l = max(0,row-window_size)
            r = min(row+window_size,I.shape[0])
            u = max(0, col-window_size)
            d = min(row+window_size,I.shape[1])
            window = I[l:r,d:u]
            if (window.size != 0):
                gauss_I = filt.gaussian(window, sigma=15, multichannel=False)
                gradient_magnitude = color_sobel_edges(gauss_I)
                oned_arr = np.reshape(window, [gradient_magnitude[0] * gradient_magnitude[1]])
                K = scipy.stats.kurtosis(oned_arr)
                sharpness = np.log(K + 3)
                shrp_arr[row/window_size+col/window_size] = sharpness

mushroom = io.imread('mushroom.jpg')
I = util.img_as_float(mushroom)
sharpness = sharpness_map(I, 100)
plot(sharpness)
plt.colorbar()
plt.show()

# Step 4: Try it on another image.

Use the functions you wrote to produce the same plots as in steps 2 and 3 but for the `waterfall.jpg` image instead.


In [ ]:
water = io.imread('waterfall.jpg')
water_I = util.img_as_float(water)
kurtosis = test_blur_measure(I, 1, 30)
plot(kurtosis[:, 0], kurtosis[:, 1])
plt.xlabel('Sigma')
plt.ylabel('Kurtosis')
plt.title('Kurtosis values for gaussian blurred gradient magnitudes with varying sigma values for waterfall.jpg')
plt.show()

sharpness = sharpness_map(water_I, 100)
plot(sharpness)
plt.colorbar()
plt.show()

# Step 5: Thinking and Qualitative Analysis

### Answer the following questions, right here in this block.

1. Do you think that the log(kurtosis+3) measurement of sharpness (hereafter called the "sharpness measure") is a good measure for characterizing global image blur (the general amount that the entire image is blurred)?  Explain and justify your answer.

	_Your answer:_  

2. Is the sharpness measure effective at characterizing variations in local blur?  Does it respond to different regions of images appropriately?  Explain and justify your answers.

	_Your answer:_  

3. Think of what the shape of a histogram of gradient magnitudes would look like for a sharp image.  Why does this set of gradient magnitudes have high kurtosis?  (It might help to look up kurtosis and see what it measures about a histogram!)

	_Your answer:_  

4. Now think what would happen as that same image gets blurrier.  Explain how the shape of the histogram would change, and the corresponding effect on the kurtosis.

	_Your answer:_  